In [26]:
import gurobipy as gp
import numpy as np

In [27]:
inst="1"
P=2 #number of projects
core=[]
pred=[]
dur=[]
recu=[]
for i in range(P):
    sample=str(i+1)
    core.append(np.loadtxt("Instancias/rcpsp/Datos30/core"+inst+sample+".txt", dtype='int'))
    pred.append(np.loadtxt("Instancias/rcpsp/Datos30/pred"+inst+sample+".txt", dtype='int'))
    dur.append(np.loadtxt("Instancias/rcpsp/Datos30/dura"+inst+sample+".txt", dtype='int'))
    recu.append(np.loadtxt("Instancias/rcpsp/Datos30/recu"+inst+sample+".txt", dtype='int'))


In [28]:
J=[]
n=[]
H=[]
d=[]
K=0
r=[]
es=[]
ls=[]
Tmax=0
for p in range(P):
    K=max(K,recu[p][-1][0]) 
for p in range(P):
    J.append([dur[p][i][0] for i in range(len(dur[p]))])
    n.append(len(J[p]))
    Hr=pred[p]-1
    d.append([dur[p][i][1] for i in range(len(dur[p]))])    
    r.append([[core[p][4*i+j][2] for j in range(K)] for i in range(n[p])])
    es.append([0 for i in range(n[p])])
    Tmax=Tmax+sum(d[p]) 
    print(sum(d[p]))   
    Pr=np.zeros((n[p],n[p]))
    for h in range(len(Hr)):
        Pr[Hr[h,0],Hr[h,1]]=1
    H.append(Pr)
K=int(K)
R=[0 for _ in range(K)]
for k in range(K):
    for p in range(P):        
        R[k]=max(R[k],recu[p][k][1])

Cmax=[]
for p in range(P):
    ls.append([Tmax for i in range(n[p])])
    Cmax.append(43)
    
c=[4 for _ in range(K)]
w=[0 for _ in range(P)]

158
160


In [29]:
for p in range(P):
    Pr=H[p]

    for j in range(n[p]):
        es[p][j] = 0
        for i in range(j):
            if Pr[i,j] == 1:
                if es[p][j] < es[p][i] + d[p][i]:
                    es[p][j] = es[p][i] + d[p][i]

    for i in range(n[p]-1,-1,-1):
        ls[p][i] = Cmax[p]
        for j in range(i+1,n[p]):
            if Pr[i,j] == 1:
                if ls[p][i] > ls[p][j] - d[p][i]:
                    ls[p][i] = ls[p][j] - d[p][i]

In [32]:
RTC=gp.Model("Resource Constrained Project Scheduling Problem")

x=RTC.addVars([(i,p,j,p2) for p in range(P)
                                 for p2 in range(P)
                                 for i in range(n[p])
                                 for j in range(n[p2])], vtype=gp.GRB.BINARY, name='x')
y=RTC.addVars([(i,p,j,p2) for p in range(P)
                                 for p2 in range(P)
                                 for i in range(n[p])
                                 for j in range(n[p2])], vtype=gp.GRB.BINARY, name='y')
S=RTC.addVars([(i,p) for p in range(P)
                        for i in range(n[p])], vtype=gp.GRB.CONTINUOUS, name='S')
h=RTC.addVars(K, vtype=gp.GRB.CONTINUOUS, name='h')

In [33]:
RTC.setObjective(sum(w[p] for p in range(P)) + sum(c[k]*h[k] for k in range(K)), gp.GRB.MINIMIZE)

RTC.addConstrs((x[i,p,j,p2]+x[j,p2,i,p] <= 1  for p in range(P)
                                    for p2 in range(P)
                                    for i in range(n[p])
                                    for j in range(n[p2])
                                    if (H[p][i,j]==0 and p==p2) or p!=p2))

RTC.addConstrs((S[j,p2]>= S[i,p] + d[p][i]-Cmax[p]*(1-x[i,p,j,p2]) for p in range(P)
                                                    for p2 in range(P)
                                                    for i in range(n[p])
                                                    for j in range(n[p2])
                                                    if i!=j or p!=p2))

RTC.addConstrs((x[i,p,j,p] == 1 for p in range(P)
                                for i in range(n[p])
                                for j in range(n[p])
                                if i<j and H[p][i,j]==1))

RTC.addConstrs((S[0,p] == 0 for p in range(P)))
RTC.addConstrs((S[n[p]-1,p] <= Cmax[p] for p in range(P)))
RTC.addConstrs((S[i,p] + d[p][i] <= Cmax[p] for p in range(P)
                                            for i in range(n[p])))

RTC.addConstrs((S[i,p] >= es[p][i] for p in range(P)
                                for i in range(n[p])))
RTC.addConstrs((S[i,p] <= ls[p][i] for p in range(P)
                                for i in range(n[p])))

RTC.addConstrs((y[i,p,j,p2]<=1-x[i,p,j,p2]-x[j,p2,i,p] for p in range(P)
                                        for p2 in range(P)
                                        for i in range(n[p])
                                        for j in range(n[p2])
                                        if i!=j or p!=p2))

RTC.addConstrs((S[j,p2]>=S[i,p]-Cmax[p]*(1-y[i,p,j,p2])  for p in range(P)
                                                        for p2 in range(P)
                                                        for i in range(n[p])
                                                        for j in range(n[p2])
                                                        if i!=j or p!=p2))

RTC.addConstrs((S[j,p2]<=S[i,p]+d[p][i]+Cmax[p]*(1-y[i,p,j,p2]) for p in range(P)
                                                            for p2 in range(P)
                                                            for i in range(n[p])
                                                            for j in range(n[p2])
                                                            if i!=j or p!=p2))

RTC.addConstrs((y[i,p,j,p2]+y[j,p2,i,p]+x[i,p,j,p2]+x[j,p2,i,p]>=1  for p in range(P)
                                                for p2 in range(P)
                                                for i in range(n[p])
                                                for j in range(n[p2])
                                                if i!=j or p!=p2))

RTC.addConstrs((r[p][i][k]+sum(r[p][j][k]*y[j,p,i,p] for j in range(n[p]) if j!=i)
                 + sum(sum(r[p2][j][k]*y[j,p2,i,p] for j in range(n[p2]))for p2 in range(P) if p2!=p)
                <=R[k] + h[k] for p in range(P)
                                                                            for i in range(n[p])
                                                                            for k in range(K)))

RTC.update

<bound method Model.update of <gurobi.Model Continuous instance Resource Constrained Project Scheduling Problem: 0 constrs, 0 vars, Parameter changes: Username=(user-defined)>>

In [34]:
RTC.setParam('OutputFlag',False)
RTC.setParam(gp.GRB.Param.TimeLimit,600)
RTC.optimize()
print(RTC.ObjVal)

36.0


In [36]:
print("Objective = ",RTC.objVal)
print("Gap = ",RTC.MIPGap*100)
print("Running time = ",RTC.Runtime)
print(h)
for v in RTC.getVars():
    if v.X > 0.1:
        print('%s: %g' % (v.varName, v.x))
#print(Cmax)

Objective =  36.0
Gap =  0.0
Running time =  30.759000062942505
{0: <gurobi.Var h[0] (value 1.0)>, 1: <gurobi.Var h[1] (value 2.0)>, 2: <gurobi.Var h[2] (value 0.0)>, 3: <gurobi.Var h[3] (value 6.0)>}
x[0,0,1,0]: 1
x[0,0,2,0]: 1
x[0,0,3,0]: 1
x[0,0,4,0]: 1
x[0,0,5,0]: 1
x[0,0,6,0]: 1
x[0,0,7,0]: 1
x[0,0,8,0]: 1
x[0,0,9,0]: 1
x[0,0,10,0]: 1
x[0,0,11,0]: 1
x[0,0,12,0]: 1
x[0,0,13,0]: 1
x[0,0,14,0]: 1
x[0,0,15,0]: 1
x[0,0,16,0]: 1
x[0,0,17,0]: 1
x[0,0,18,0]: 1
x[0,0,19,0]: 1
x[0,0,20,0]: 1
x[0,0,21,0]: 1
x[0,0,22,0]: 1
x[0,0,23,0]: 1
x[0,0,24,0]: 1
x[0,0,25,0]: 1
x[0,0,26,0]: 1
x[0,0,27,0]: 1
x[0,0,28,0]: 1
x[0,0,29,0]: 1
x[0,0,30,0]: 1
x[0,0,31,0]: 1
x[1,0,4,0]: 1
x[1,0,5,0]: 1
x[1,0,8,0]: 1
x[1,0,10,0]: 1
x[1,0,11,0]: 1
x[1,0,13,0]: 1
x[1,0,14,0]: 1
x[1,0,15,0]: 1
x[1,0,16,0]: 1
x[1,0,17,0]: 1
x[1,0,18,0]: 1
x[1,0,19,0]: 1
x[1,0,20,0]: 1
x[1,0,21,0]: 1
x[1,0,22,0]: 1
x[1,0,23,0]: 1
x[1,0,24,0]: 1
x[1,0,25,0]: 1
x[1,0,26,0]: 1
x[1,0,27,0]: 1
x[1,0,28,0]: 1
x[1,0,29,0]: 1
x[1,0,30,0]: 1
x